In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')
Path("../tests/test1/d10").mkdir(parents=True, exist_ok=True)
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

Время последнего моделирования: 23/03/2021 03:03


In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 10
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
results = []
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']
precisions_3 = ['{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.2f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.4f}', '{0:.4f}', '{0:.1f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    algorithm_name = 'ММП v2'
    res = [algorithm_name]
    dict_results = tma.swarm(algorithm_name = algorithm_name, n=100, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    nf_mean = df['Nf'].mean()
    nf_max = max(df['Nf'])
    iter_mean = df['Iter'].mean()
    iter_max = max(df['Iter'])
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(nf_mean, nf_max))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(iter_mean, iter_max))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1).head())

    res.extend([k_a_mean, nf_mean, nf_max, iter_mean, iter_max, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d10/ЛМ, ' + str(std) + '.xlsx')
    print('_________________________________________________')

df_lm = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
df_lm['СКО'] = std_arr
results = []

Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 1.3 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0059, t_max = 0.0237


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,10.000,98.786,11.58,70.56,15.807,70.56,15.807,0.00,"[1, 1, 1, 1]",0.0196,19,15
1,0.0,10.000,75.064,11.64,0.0,10.000,75.064,11.64,53.18,18.328,53.18,18.328,0.00,"[1, 1, 1, 1]",0.0067,12,10
2,0.0,10.000,78.479,3.31,360.0,10.000,78.479,3.31,31.77,9.618,31.77,9.618,0.00,"[1, 1, 1, 1]",0.0045,8,7
3,0.0,10.000,99.144,11.50,360.0,10.000,99.144,11.50,70.66,15.677,70.66,15.677,0.00,"[1, 1, 1, 1]",0.0076,19,15
4,0.0,10.000,174.065,9.57,0.0,10.000,174.065,9.57,149.70,4.673,149.70,4.673,0.00,"[1, 1, 1, 1]",0.0162,24,18


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.9 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0039, t_max = 0.0092


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,10.026,98.804,11.63,70.56,15.807,70.58,15.864,0.01,"[1, 1, 1, 1]",0.0065,19,15
1,0.0,10.000,75.064,11.64,0.0,9.965,75.028,11.59,53.18,18.328,53.16,18.247,0.01,"[1, 1, 1, 1]",0.0028,12,10
2,0.0,10.000,78.479,3.31,0.0,10.008,78.788,3.30,31.77,9.618,31.78,9.600,0.01,"[1, 1, 1, 1]",0.0020,8,7
3,0.0,10.000,99.144,11.50,0.0,9.982,99.167,11.46,70.66,15.677,70.66,15.628,0.01,"[1, 1, 1, 1]",0.0045,19,15
4,0.0,10.000,174.065,9.57,360.0,10.016,174.042,9.59,149.70,4.673,149.68,4.682,0.01,"[1, 1, 1, 1]",0.0074,24,18


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.9 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.04
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0041, t_max = 0.0118


...

_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 1.0 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 95%, Рэф_мин = 0
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.25
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0043, t_max = 0.0131


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.9,10.134,98.876,11.84,70.56,15.807,70.67,16.092,0.25,"[1, 1, 1, 1]",0.0044,19,15
1,0.0,10.000,75.064,11.64,0.0,9.830,74.882,11.36,53.18,18.328,53.08,17.928,0.25,"[1, 1, 1, 1]",0.0030,12,10
2,0.0,10.000,78.479,3.31,0.1,10.037,80.045,3.25,31.77,9.618,31.83,9.529,0.26,"[1, 1, 1, 1]",0.0020,8,7
3,0.0,10.000,99.144,11.50,0.1,9.910,99.260,11.29,70.66,15.677,70.65,15.433,0.26,"[1, 1, 1, 1]",0.0043,19,15
4,0.0,10.000,174.065,9.57,360.0,10.080,173.947,9.66,149.70,4.673,149.59,4.717,0.23,"[1, 1, 1, 1]",0.0091,24,18


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.9 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 75%, Рэф_мин = 0
Вариант 1: Рэф = 95%, Рэф_мин = 0
Вариант 2: Рэф = 96%, Рэф_мин = 0
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.99
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0042, t_max = 0.0125


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.8,10.271,98.965,12.10,70.56,15.807,70.78,16.384,1.00,"[1, 1, 1, 1]",0.0125,19,15
1,0.0,10.000,75.064,11.64,0.1,9.668,74.699,11.09,53.18,18.328,52.99,17.545,0.99,"[1, 1, 1, 1]",0.0099,12,10
2,0.0,10.000,78.479,3.31,0.1,10.074,81.664,3.19,31.77,9.618,31.90,9.439,1.05,"[1, 1, 1, 1]",0.0075,8,7
3,0.0,10.000,99.144,11.50,0.2,9.821,99.377,11.09,70.66,15.677,70.64,15.192,1.05,"[1, 1, 1, 1]",0.0067,19,15
4,0.0,10.000,174.065,9.57,359.9,10.162,173.828,9.76,149.70,4.673,149.47,4.762,0.94,"[1, 1, 1, 1]",0.0061,24,18


_________________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    algorithm_name = 'Метод N пеленгов'
    res = [algorithm_name]
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results).dropna(axis=1)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))
    
    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1).head())

    res.extend([k_a_mean, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d10/N, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_________________________________________________')

df_n = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 't', 't_max'])
df_n['СКО'] = std_arr

Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.6 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
t = 0.0004, t_max = 0.0014


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,10.000,98.786,11.58,0.0,10.000,98.786,11.58,70.6,15.807,70.561,15.81,0.00,"[1, 1, 1, 1]",0.001
1,0.0,10.000,75.064,11.64,0.0,10.000,75.064,11.64,53.2,18.328,53.176,18.33,0.00,"[1, 1, 1, 1]",0.001
2,0.0,10.000,78.479,3.31,0.0,10.000,78.479,3.31,31.8,9.618,31.768,9.62,0.00,"[1, 1, 1, 1]",0.001
3,0.0,10.000,99.144,11.50,0.0,10.000,99.144,11.50,70.7,15.677,70.663,15.68,0.00,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,0.0,10.000,174.065,9.57,149.7,4.673,149.704,4.67,0.00,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.6 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 96%, Рэф_мин = 0
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
t = 0.0004, t_max = 0.0017


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,10.000,98.786,11.58,359.8,10.151,98.690,11.91,70.6,15.807,70.597,16.18,0.01,"[1, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,359.8,9.999,74.867,11.77,53.2,18.328,53.162,18.47,0.01,"[1, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,9.995,79.076,3.27,31.8,9.618,31.763,9.55,0.01,"[1, 1, 1, 1]",0.000
3,0.0,10.000,99.144,11.50,359.8,10.118,98.991,11.77,70.7,15.677,70.670,15.99,0.01,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,360.0,9.999,174.061,9.56,149.7,4.673,149.699,4.67,0.01,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 82%, Рэф_мин = 0
Вариант 1: Рэф = 96%, Рэф_мин = 0
Вариант 2: Рэф = 97%, Рэф_мин = 0
Вариант 3: Рэф = 99%, Рэф_мин = 0
Ка = 0.04
t = 0.0004, t_max = 0.0015


...

_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 9%, Рэф_мин = 0
Вариант 1: Рэф = 43%, Рэф_мин = 0
Вариант 2: Рэф = 44%, Рэф_мин = 0
Вариант 3: Рэф = 69%, Рэф_мин = 0
Ка = 0.31
t = 0.0004, t_max = 0.0013


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,10.000,98.786,11.58,359.1,9.110,97.028,10.69,70.6,15.807,70.309,14.61,0.38,"[0, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,359.2,9.003,72.950,10.76,53.2,18.328,52.766,16.81,0.36,"[0, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.2,9.734,81.388,2.94,31.8,9.618,31.535,8.97,0.27,"[0, 0, 0, 1]",0.000
3,0.0,10.000,99.144,11.50,359.1,8.864,96.977,10.24,70.7,15.677,70.228,14.06,0.46,"[0, 0, 0, 1]",0.000
4,0.0,10.000,174.065,9.57,360.0,9.759,174.290,9.26,149.7,4.673,149.886,4.53,0.25,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 2%, Рэф_мин = 0
Вариант 2: Рэф = 2%, Рэф_мин = 0
Вариант 3: Рэф = 4%, Рэф_мин = 0
Ка = 1.74
t = 0.0003, t_max = 0.0011


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,10.000,98.786,11.58,358.2,5.751,89.933,5.83,70.6,15.807,68.542,8.58,3.09,"[0, 0, 0, 0]",0.000
1,0.0,10.000,75.064,11.64,358.4,6.611,67.437,7.56,53.2,18.328,51.393,11.97,2.17,"[0, 0, 0, 0]",0.000
2,0.0,10.000,78.479,3.31,0.4,8.991,84.520,2.27,31.8,9.618,30.747,7.72,1.22,"[0, 0, 0, 0]",0.001
3,0.0,10.000,99.144,11.50,358.2,5.511,89.020,5.42,70.7,15.677,68.188,8.08,3.68,"[0, 0, 0, 0]",0.001
4,0.0,10.000,174.065,9.57,359.9,9.014,175.156,8.33,149.7,4.673,150.621,4.09,1.12,"[0, 0, 0, 1]",0.000


_________________________________________________


In [7]:
df = pd.concat([df_lm, df_n])
for i, j in enumerate(df.columns):
        try: df[j] = df[j].apply(lambda x: precisions_3[i].format(np.float64(x)))
        except(ValueError): pass

print('Сводная таблица:')
df.groupby('СКО').apply(lambda a: a.drop('СКО', axis=1)[:]).reset_index().drop('level_1', axis=1)

Сводная таблица:


,СКО,Алгоритм,В11,В12,В21,В22,В31,В32,В41,В42,Ка,Nf,Nf_max,Iter,Iter_max,t,t_max
0,0.0,ММП v2,100,1,100,1,100,1,100,1,0.00,15,27,12,21,0.0059,0.0237
1,0.0,Метод N пеленгов,100,1,100,1,100,1,100,1,0.00,nan,nan,nan,nan,0.0004,0.0014
2,0.1,ММП v2,100,1,100,1,100,1,100,1,0.01,15,27,12,21,0.0039,0.0092
3,0.1,Метод N пеленгов,96,0,100,1,100,1,100,1,0.01,nan,nan,nan,nan,0.0004,0.0017
4,0.2,ММП v2,100,1,100,1,100,1,100,1,0.04,15,27,12,21,0.0041,0.0118
5,0.2,Метод N пеленгов,82,0,96,0,97,0,99,0,0.04,nan,nan,nan,nan,0.0004,0.0015
6,0.3,ММП v2,100,1,100,1,100,1,100,1,0.09,15,27,12,21,0.0040,0.0076
7,0.3,Метод N пеленгов,57,0,84,0,84,0,94,0,0.10,nan,nan,nan,nan,0.0004,0.0012
8,0.5,ММП v2,95,0,100,1,100,1,100,1,0.25,15,27,12,21,0.0043,0.0131
9,0.5,Метод N пеленгов,9,0,43,0,44,0,69,0,0.31,nan,nan,nan,nan,0.0004,0.0013
